In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import numpy as np 
import skimage
from skimage import transform
from skimage import io 
from matplotlib import pyplot as plt
%matplotlib inline

/Users/david.buchaca/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import keras

In [3]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
import keras

In [6]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


INFO:plaidml:Opening device "metal_intel_iris_pro_graphics.0"


In [7]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
%%time 
epochs = 1
model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/1


INFO:plaidml:Analyzing Ops: 127 of 320 operations complete
ERROR:plaidml:Compiler::Build> Compilation failure:
Compilation failed: 

program_source:45:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:142:7: warning: unused variable 'tid'
  int tid = _tid;
      ^
program_source:3739:9: warning: unused variable 'x0_tid'
    int x0_tid = (tid % 32);
        ^
program_source:10701:12: error: cannot initialize a variable of type 'int2' (aka 'vector_int2') with an rvalue of type 'vec<char, 2>' (vector of 2 'char' values)
      int2 LX_T388 = select((char2)((float2)0), (char2)((float2)1), ((bool2)LX_T387));
           ^         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



Internal: The exception chain appears to be corrupt

In [9]:
# 3 ms step